In [1]:
import cv2
import mediapipe as mp
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
import random
from itertools import count
from matplotlib.animation import FuncAnimation
import time
import psutil
import matplotlib.pyplot as plt
import random
from itertools import count
from matplotlib.animation import FuncAnimation
plt.style.use('fivethirtyeight')

In [3]:
# dataf = pd.read_csv('FinalData.csv')
dataf = pd.read_csv('FinalData.csv', header=None)

# Convert dataframe to NumPy array
arr = dataf.to_numpy()

# Transpose the array so that rows become columns
# arr = np.transpose(arr)

# Create a list of arrays, where each array represents a row
rows = []
for i in range(arr.shape[0]):
    row = arr[i]
    rows.append(row)



In [4]:
np.array(rows).shape

(100, 2112)

In [5]:
NewTrainingArray = np.array(rows)
(NewTrainingArray).shape

(100, 2112)

In [6]:
y_train = []
No_of_Stroke_Videos = 50  
for i in range(No_of_Stroke_Videos):
    y_train.append(1)
len(y_train)

50

In [7]:
No_of_No_Stroke_Videos = 50
for i in range(No_of_No_Stroke_Videos):
    y_train.append(0)
len(y_train)

100

In [8]:
y_train = np.array(y_train)
y_train=y_train[:,np.newaxis]
print(y_train)

[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [9]:
def trimTestArray(training_array):
    arr_Temp = np.array(training_array, dtype="object")
    # Find the minimum number of columns across all rows
    min_cols = np.array(rows).shape[1]
    # Remove the last excess element from each row that has more columns than the minimum number
    trimmed_arr = np.array([row[:min_cols] for row in arr_Temp])
    len(trimmed_arr[0])
    return trimmed_arr


# LSTM MODEL

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Load your flattened data into X and labels into y
X = np.array(NewTrainingArray, dtype="float")
X = X.reshape(X.shape[0], 1, X.shape[1])

y = np.array(y_train, dtype="float")

# Define the LSTM model architecture with L2 regularization and early stopping
model = Sequential()
model.add(LSTM(128, input_shape=(1, np.array(rows).shape[1]), kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model with early stopping
history = model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
model.fit(X, y, epochs=1000, batch_size=32, validation_split=0.2)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               1147392   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,147,521
Trainable params: 1,147,521
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


# 1D CNN MODEL

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load your flattened data into X and labels into y
X = np.array(NewTrainingArray, dtype="float")
y = np.array(y_train, dtype="float")

# Define the CNN model architecture
modell = Sequential()
modell.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(np.array(rows).shape[1], 1)))
modell.add(MaxPooling1D(pool_size=2))
modell.add(Dropout(0.2))
modell.add(Flatten())
modell.add(Dense(1, activation='sigmoid'))

# Compile the model with binary cross-entropy loss and Adam optimizer
modell.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modell.summary()
# Define callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Fit the model with callbacks
history = modell.fit(X, y, epochs=1000, batch_size=32, validation_split=0.3)


# 12D CNN

In [1]:
import h5py
import numpy as np

# Open the HDF5 file in read mode
with h5py.File('train_file.h5', 'r') as f:

    # Load the dataset into a NumPy array
    train_array = np.array(f['my_dataset'])

# Print the shape of the array to verify it was loaded correctly
print(train_array.shape[0])

80


In [2]:
(train_array).shape

(80, 176, 12)

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten,Dropout

In [4]:
No_of_Stroke_Videos = 40 
x_train = train_array
y_train=np.concatenate((np.ones(No_of_Stroke_Videos), np.zeros(No_of_Stroke_Videos)))


print(y_train)
input_shape = ((train_array).shape[1], 12)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [5]:
from keras.callbacks import EarlyStopping

# Define the number of classes (2 for binary classification)
num_classes = 1

# Define the model architecture
CNNmodel = Sequential()
CNNmodel.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
CNNmodel.add(MaxPooling1D(pool_size=2))
CNNmodel.add(Flatten())
CNNmodel.add(Dense(64, activation='relu'))
CNNmodel.add(Dropout(0.4))
CNNmodel.add(Dense(1, activation='sigmoid'))

# Compile the model
CNNmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
CNNmodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 174, 32)           1184      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 87, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 2784)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                178240    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 179,489
Trainable params: 179,489
Non-trainable params: 0
__________________________________________________

In [ ]:
# Train the model with early stopping
CNNmodel.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10


# Checking The Model 

In [ ]:
testing_array = []

In [ ]:
import time
# Final Testing 
def Test_final_CNN():
    z = 0
    string = ""
    # TRACKING THE HANDS
    L_shoulder_array=[]
    L_elbow_array=[]
    L_wrist_array=[]
    R_shoulder_array=[]

    R_elbow_array=[]
    R_wrist_array=[]


    

    cap = cv2.VideoCapture(0)
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark

                # Get coordinates

                #Left Hand
                L_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                L_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                L_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                #Right Hand

                R_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                R_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                R_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]


                # Putting coordinates in to arrays

                #Left Hand

    #             L_shoulder_array.append(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x)
    #             L_elbow_array.append(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x)
    #             L_wrist_array.append(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x)

                #Right Hand

    #             R_shoulder_array.append(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x)
    #             R_elbow_array.append(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x)
    #             R_wrist_array.append(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x)


                
                # import the time module
                
                # define the countdown func.
                tabx1 = []
                taby1 = []
            
                tabx2 = []
                taby2 = []
            
                tabx3 = []
                taby3 = []
            
                tabx4 = []
                taby4 = []
            
                tabx5 = []
                taby5 = []
            
                tabx6 = []
                taby6 = []
                testing_array = []   
                t = 180
                while t:
                    
                    tabx1.append(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x)
                    tabx2.append(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x)
                    tabx3.append(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x)
                    tabx4.append(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x)
                    tabx5.append(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x)
                    tabx6.append(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x)

                    taby1.append(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y) 
                    taby2.append(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y) 
                    taby3.append(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y) 
                    taby4.append(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y) 
                    taby5.append(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y) 
                    taby6.append(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
#                     time.sleep(1)
                    t -= 1
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
                
                df = pd.DataFrame(list(zip(tabx1,tabx2,tabx3,tabx4,tabx5,tabx6,taby1,taby2,taby3,taby4,taby5,taby6)),columns =['L_shoulder_x','L_elbow_x','L_wrist_x','R_shoulder_x','R_elbow_x','R_wrist_x','L_shoulder_y','L_elbow_y','L_wrist_y','R_shoulder_y','R_elbow_y','R_wrist_y'])
                flattened_DataFrame = df.values.flatten()
                testing_array.append(list(flattened_DataFrame))
                testing_array = trimTestArray(testing_array)
#                 print('test shape',testing_array.shape)
                X_test = np.array(testing_array, dtype="float")
                
                # Reshape X_test to the correct shape
#                 X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
                
                # Make predictions on the test data
                y_pred = modell.predict(X_test)
                # Convert the predicted probabilities to binary predictions
                y_pred_binary = (y_pred > 0.5).astype(int)
                
                if z ==40:
                    if y_pred_binary[0][0]==0:
                        string = "No Stroke"
                    else:
                        string = "Stroke"
                    z = 0
                z +=1
#                 print('z-',z)
                
                if string == "No Stroke":
                    
                    cv2.putText(image, string, tuple(np.multiply(0.3, [100, 48]).astype(int)), cv2.FONT_ITALIC, 0.5, (0,255,0), 2, cv2.LINE_AA)
                else:
                    cv2.putText(image, string, tuple(np.multiply(0.3, [100, 48]).astype(int)), cv2.FONT_ITALIC, 0.5, (0,25,255), 2, cv2.LINE_AA)

#                 print(f"Predicted label: {y_pred_binary}")
                
                
                '''
                
                if y_pred_binary[0][0]==1:
                    string_temp = "Stroke"
                else:
                    string_temp = "No Stroke"
                    
                cv2.putText(image, string_temp, 
                               tuple(np.multiply(0.25, [60, 48]).astype(int)), 
                               cv2.FONT_ITALIC, 0.5, (0,25,255), 2, cv2.LINE_AA
                                    )
#                 print(f"Predicted label: {y_pred_binary}")
                    '''
                                





                # Visualize coordinates on the screen

#                 #Left Hand
#                 cv2.putText(image, str(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x), 
#                                tuple(np.multiply(L_shoulder, [640, 480]).astype(int)), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
#                                     )
#                 cv2.putText(image, str(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x), 
#                                tuple(np.multiply(L_elbow, [640, 480]).astype(int)), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
#                                     )
#                 cv2.putText(image, str(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x), 
#                                tuple(np.multiply(L_wrist, [640, 480]).astype(int)), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
#                                     )

#                 #Right Hand

#                 cv2.putText(image, str(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x), 
#                                tuple(np.multiply(R_shoulder, [640, 480]).astype(int)), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
#                                     )
#                 cv2.putText(image, str(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x), 
#                                tuple(np.multiply(R_elbow, [640, 480]).astype(int)), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
#                                     )
#                 cv2.putText(image, str(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x), 
#                                tuple(np.multiply(R_wrist, [640, 480]).astype(int)), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
#                                     )

            except:
                pass


            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )               

            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
#         df = pd.DataFrame(list(zip(tabx1,tabx2,tabx3,tabx4,tabx5,tabx6,taby1,taby2,taby3,taby4,taby5,taby6)),columns =['L_shoulder_x','L_elbow_x','L_wrist_x','R_shoulder_x','R_elbow_x','R_wrist_x','L_shoulder_y','L_elbow_y','L_wrist_y','R_shoulder_y','R_elbow_y','R_wrist_y'])
#         df
#         flattened_DataFrame = df.values.flatten()
        
#         global testing_array 
#         testing_array.append(list(flattened_DataFrame))
#         print('Success',df)
        

    # plotter(shoulder_array)
    # plotter(elbow_array)
    # plotter(wrist_array)


In [ ]:
Test_final_CNN()